In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# Load image and convert to tensor
def load_image(path):
    img = Image.open(path).resize((256, 256))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x[:3, :, :]),  # keep only RGB
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    return transform(img).unsqueeze(0)

# Convert tensor to image
def save_image(tensor, filename):
    img = tensor.squeeze(0).detach()
    img = img * 0.5 + 0.5  # unnormalize
    img = transforms.ToPILImage()(img)
    img.save(filename)

# Load VGG19 model
vgg = models.vgg19(pretrained=True).features.eval()

# Load content and style
content = load_image("content.jpg")
style = load_image("style.jpg")
target = content.clone().requires_grad_(True)

# Define optimizer
optimizer = torch.optim.Adam([target], lr=0.01)

# Get features
def get_features(x):
    layers = [0, 5, 10, 19, 21]
    feats = []
    for i, layer in enumerate(vgg):
        x = layer(x)
        if i in layers:
            feats.append(x)
    return feats

# Gram matrix
def gram(x):
    b, c, h, w = x.size()
    x = x.view(c, h * w)
    return torch.mm(x, x.t())

# Training
for step in range(101):
    target_feats = get_features(target)
    content_feats = get_features(content)
    style_feats = get_features(style)

    content_loss = torch.mean((target_feats[3] - content_feats[3]) ** 2)
    style_loss = 0
    for t, s in zip(target_feats, style_feats):
        style_loss += torch.mean((gram(t) - gram(s)) ** 2)

    loss = content_loss + 0.001 * style_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 20 == 0:
        print("Step", step, "Loss:", loss.item())

# Save result
save_image(target, "output.jpg")
print("Saved as output.jpg")

